# Viewing CT Scan Data In Unity
### By Jacob Hofer

In the medical field, it can be useful to obtain a 3D mesh representation of data from MRI or CT Scans. Obtaining this mesh can be done in a variety of ways, but the algorithm we will look at is marching cubes.

## Mesh Creation Pipeline
The general pipeline of generating a 3D mesh is shown below:

![PipelineImage](./NotebookImages/Pipeline.png)

In the pipeline, data is converted from `.nrrd` to a simpler `.txt` format to be loaded into Unity. Within Unity, marching cubes is ran on various function values, denoted `value1`, `value2`, and `value3` in the diagram, to obtain multiple meshes of the data.

## Converting the .nrrd File
We begin with an `.nrrd` file. In our case, the file [CTA-cardio.nrrd](./CTA-cardio.nrrd) will be used.

In order to work with our Unity scripts, we first need to convert the file to a custom `.txt` format. This will be done via Python.

As usual, we begin with importing our libraries. The only library we import is `nrrd`, installed with: 

```sh
pip install pynrrd
```


In [8]:
import nrrd

And next we tell Python what file to use. In our case [CTA-cardio.nrrd](./CTA-cardio.nrrd).

In [9]:
filename = 'CTA-cardio.nrrd'

Check that the file is read properly.

In [10]:
readdata, header = nrrd.read(filename)
print(readdata.shape)
print(header)

(512, 512, 321)
OrderedDict([('type', 'short'), ('dimension', 3), ('space', 'left-posterior-superior'), ('sizes', array([512, 512, 321])), ('space directions', array([[0.933594, 0.      , 0.      ],
       [0.      , 0.933594, 0.      ],
       [0.      , 0.      , 1.25    ]])), ('kinds', ['domain', 'domain', 'domain']), ('endian', 'little'), ('encoding', 'gzip'), ('space origin', array([-238.533267, -238.533267, -200.      ]))])


Open a file for writing the data to. In our case we can just call it `data.txt`.

In [11]:
out_file = open('data.txt', 'w')

Then we will write the data in a format we can easily read back into Unity later. Since there is a lot of data, this may take a while.

In [12]:
out_file.write(str(readdata.shape[0]) + '\n')
out_file.write(str(readdata.shape[1]) + '\n')
out_file.write(str(readdata.shape[2]) + '\n')

out_file.write('\n')

for x in range(readdata.shape[0]):
    for y in range(readdata.shape[1]):
        for z in range(readdata.shape[2]):
            out_file.write(str(readdata[x][y][z]))
            if (z != readdata.shape[2] - 1 or y != readdata.shape[1] - 1):
                out_file.write(',')
            else:
                out_file.write('\n')

Finally, we close the file, and our conversion is done!

In [13]:
out_file.close()

## Configuring Unity
Opening the provided [SampleScene.unity](./Assets/Scenes/SampleScene.unity) in Unity presents us with what appears to be an empty scene.

However, notice we have a few `GameObjects` in our scene, notably the `DataController` and the `Mesh ####`s: 

![SceneObjects](./NotebookImages/SceneObjects.png)

Inspecting the `DataController`, we see a field labeled `File Name`. We use this to tell Unity what `.txt` file to use for loading our raw function data. 

![DataController](./NotebookImages/DataController.png)

Inspecting any of the `Mesh ####` objects, we see the `Data Mesher` script defining various fields.

![Mesh](./NotebookImages/Mesh.png)

`Resolution Scale` describes the scale to divide the marching cubes resolution by. A value of 1 will perform marching cubes for every value in our data file. A value of 10 will perform marching cubes once for every 10\*10\*10=1000 values in our data file.

`Surface Value` describes the function value to generate a surface for. In the example data, the function ranges in `[-1024, 1024]`, so any value in that range will generate a surface. Each mesh object in the scene has a different value.

`Data Controller` describes which object to obtain the data from. In our scene, we only have the one shown previously, but this allows for loading multiple data files at once.

`Transform` describes the position and scaling of the output mesh. It can be useful to modify these values if the meshes overlap or display too big or small.

## Running Unity
Pressing the "play" button will have Unity generate the various meshes and add them into the scene. It is then easiest to navigate around using the scene view (as opposed to the game view) to investigate each of the meshes.

As we can see, different function values provide different meshes, with different details. For example, the function value of `200` provides a great look at the patient's skeleton:

![Skeleton](./NotebookImages/Skeleton.png)

A function value of `-300` allows us to view the patient's lungs and other internal organs:

![Lungs](./NotebookImages/Lungs.png)


## Unity Scripts
All the Unity C# scripts are located in the [Assets/Scripts](./Assets/Scripts/) directory. Each script is commented using C# XML format, and are summarized here.

[DataController.cs](./Assets/Scripts/DataController.cs) initializes an instance of `DataLoader`, passing along the file name obtained from within the Unity Editor.

[DataLoader.cs](./Assets/Scripts/DataLoader.cs) handles actually loading the data from the specified file into a 3D `float` array that can be retrieved multiple times. This prevents loading the data once for every mesh.

[DataMesher.cs](./Assets/Scripts/DataMesher.cs) handles obtaining the data from the `DataLoader` and passing it along to `MarchingCubes`. It then takes the result from `MarchingCubes` and generates a mesh using Unity's API.

[MarchingCubes.cs](./Assets/Scripts/MarchingCubes.cs) is an implementation of the marching cubes algorithm.

## CT Data Reference
The CT Scan data was sourced from [https://www.slicer.org/wiki/File:CTA-cardio.nrrd](https://www.slicer.org/wiki/File:CTA-cardio.nrrd).